# Gerador do Dataset WS100

Este notebook cria um novo dataset WS100 a partir do arquivo WS.csv, contendo apenas as colunas 'DT' e 'SODAR-100', e preenche gaps temporais com valores NaN.

In [1]:
# Importação das bibliotecas necessárias
import pandas as pd
import numpy as np
from pathlib import Path
from datetime import timedelta

print("Bibliotecas importadas com sucesso.")

Bibliotecas importadas com sucesso.


In [2]:
# Definição dos caminhos dos arquivos
input_file = Path('../converted_csv_data/WS.csv')
output_file = Path('../converted_csv_data/WS100.csv')

print(f"Arquivo de entrada: {input_file}")
print(f"Arquivo de saída: {output_file}")
print(f"Arquivo de entrada existe: {input_file.exists()}")

Arquivo de entrada: ../converted_csv_data/WS.csv
Arquivo de saída: ../converted_csv_data/WS100.csv
Arquivo de entrada existe: True


In [3]:
# Carregamento do dataset WS.csv
print("Carregando o dataset WS.csv...")
df_ws = pd.read_csv(input_file)

print(f"Shape do dataset original: {df_ws.shape}")
print(f"Colunas disponíveis: {list(df_ws.columns)}")
print("\nPrimeiras 5 linhas:")
df_ws.head()

Carregando o dataset WS.csv...
Shape do dataset original: (41786, 27)
Colunas disponíveis: ['DT', 'SODAR-30', 'SODAR-40', 'SODAR-50', 'SODAR-60', 'SODAR-70', 'SODAR-80', 'SODAR-90', 'SODAR-100', 'SODAR-110', 'SODAR-120', 'SODAR-130', 'SODAR-140', 'SODAR-150', 'SODAR-160', 'SODAR-170', 'SODAR-180', 'SODAR-190', 'SODAR-200', 'SODAR-210', 'SODAR-220', 'SODAR-230', 'SODAR-240', 'SODAR-250', 'SODAR-260', 'SODAR-270', 'SODAR-280']

Primeiras 5 linhas:


,DT,SODAR-30,SODAR-40,SODAR-50,SODAR-60,SODAR-70,SODAR-80,SODAR-90,SODAR-100,SODAR-110,...,SODAR-190,SODAR-200,SODAR-210,SODAR-220,SODAR-230,SODAR-240,SODAR-250,SODAR-260,SODAR-270,SODAR-280
0,2021/11/10 17:20:00,4.45,5.03,5.53,5.54,5.66,5.74,5.82,5.83,5.79,...,5.39,5.41,5.43,5.45,5.41,5.50,5.61,5.51,5.33,5.45
1,2021/11/10 17:30:00,4.04,4.94,5.30,5.29,5.72,5.87,5.98,5.99,6.06,...,6.64,6.70,6.73,6.60,6.47,6.24,6.45,6.80,7.03,7.29
2,2021/11/10 17:40:00,3.65,4.44,5.96,5.66,6.44,6.55,6.61,6.67,6.67,...,6.64,6.76,6.54,6.70,6.52,6.56,6.60,6.54,7.00,7.24
3,2021/11/10 17:50:00,2.96,4.28,4.91,5.04,5.80,5.95,6.03,6.15,6.17,...,6.35,6.30,6.15,6.31,6.70,7.16,7.24,7.06,6.86,6.56
4,2021/11/10 18:00:00,3.08,4.53,5.02,4.66,5.52,5.62,5.70,5.90,5.93,...,6.48,6.62,6.58,6.52,6.54,6.47,6.46,6.64,6.64,6.95


In [4]:
# Criação do dataset WS100 com apenas as colunas DT e SODAR-100
print("Criando dataset WS100 com colunas DT e SODAR-100...")

# Verificação se as colunas existem
required_columns = ['DT', 'SODAR-100']
missing_columns = [col for col in required_columns if col not in df_ws.columns]

if missing_columns:
    raise ValueError(f"Colunas não encontradas: {missing_columns}")

# Seleção das colunas necessárias
df_ws100 = df_ws[['DT', 'SODAR-100']].copy()

print(f"Shape do dataset WS100: {df_ws100.shape}")
print("\nPrimeiras 5 linhas do WS100:")
df_ws100.head()

Criando dataset WS100 com colunas DT e SODAR-100...
Shape do dataset WS100: (41786, 2)

Primeiras 5 linhas do WS100:


,DT,SODAR-100
0,2021/11/10 17:20:00,5.83
1,2021/11/10 17:30:00,5.99
2,2021/11/10 17:40:00,6.67
3,2021/11/10 17:50:00,6.15
4,2021/11/10 18:00:00,5.90


In [5]:
# Conversão da coluna DT para datetime
print("Convertendo coluna DT para formato datetime...")
df_ws100['DT'] = pd.to_datetime(df_ws100['DT'])

# Ordenação por data para garantir sequência correta
df_ws100 = df_ws100.sort_values('DT').reset_index(drop=True)

print(f"Primeira data: {df_ws100['DT'].min()}")
print(f"Última data: {df_ws100['DT'].max()}")
print(f"Total de registros: {len(df_ws100)}")

Convertendo coluna DT para formato datetime...
Primeira data: 2021-11-10 17:20:00
Última data: 2022-10-09 08:30:00
Total de registros: 41786


In [6]:
# Análise dos gaps temporais existentes
print("Analisando gaps temporais...")

# Cálculo das diferenças entre timestamps consecutivos
time_diffs = df_ws100['DT'].diff()

# Intervalo esperado de 10 minutos
expected_interval = timedelta(minutes=10)

# Identificação de gaps (diferenças maiores que 10 minutos)
gaps = time_diffs[time_diffs > expected_interval]

print(f"Número de gaps encontrados: {len(gaps)}")
print(f"Maior gap: {gaps.max() if len(gaps) > 0 else 'Nenhum'}")

if len(gaps) > 0:
    print("\nPrimeiros 10 gaps encontrados:")
    for i, (idx, gap) in enumerate(gaps.head(10).items()):
        start_time = df_ws100.loc[idx-1, 'DT']
        end_time = df_ws100.loc[idx, 'DT']
        missing_points = int(gap.total_seconds() / 600) - 1  # 600 segundos = 10 minutos
        print(f"{i+1}. Entre {start_time} e {end_time}: {gap} ({missing_points} pontos faltando)")

Analisando gaps temporais...
Número de gaps encontrados: 34
Maior gap: 15 days 02:20:00

Primeiros 10 gaps encontrados:
1. Entre 2021-11-13 03:40:00 e 2021-11-13 08:10:00: 0 days 04:30:00 (26 pontos faltando)
2. Entre 2021-11-20 07:40:00 e 2021-11-20 08:40:00: 0 days 01:00:00 (5 pontos faltando)
3. Entre 2021-12-02 20:50:00 e 2021-12-03 01:10:00: 0 days 04:20:00 (25 pontos faltando)
4. Entre 2022-01-21 04:50:00 e 2022-01-21 12:00:00: 0 days 07:10:00 (42 pontos faltando)
5. Entre 2022-02-19 03:20:00 e 2022-02-20 00:10:00: 0 days 20:50:00 (124 pontos faltando)
6. Entre 2022-02-22 18:30:00 e 2022-02-22 18:50:00: 0 days 00:20:00 (1 pontos faltando)
7. Entre 2022-02-23 00:30:00 e 2022-02-25 10:40:00: 2 days 10:10:00 (348 pontos faltando)
8. Entre 2022-03-05 00:20:00 e 2022-03-06 00:10:00: 0 days 23:50:00 (142 pontos faltando)
9. Entre 2022-03-08 11:40:00 e 2022-03-08 13:50:00: 0 days 02:10:00 (12 pontos faltando)
10. Entre 2022-03-08 13:50:00 e 2022-03-09 00:10:00: 0 days 10:20:00 (61 ponto

In [7]:
# Preenchimento dos gaps temporais com valores NaN
print("Preenchendo gaps temporais com valores NaN...")

# Criação de uma série temporal completa com intervalos de 10 minutos
start_time = df_ws100['DT'].min()
end_time = df_ws100['DT'].max()

print(f"Criando série temporal completa de {start_time} até {end_time}...")

# Geração de todos os timestamps com intervalo de 10 minutos
complete_timeline = pd.date_range(start=start_time, end=end_time, freq='10T')

print(f"Timestamps esperados: {len(complete_timeline)}")
print(f"Timestamps existentes: {len(df_ws100)}")
print(f"Timestamps faltando: {len(complete_timeline) - len(df_ws100)}")

# Criação do dataframe completo
df_complete = pd.DataFrame({'DT': complete_timeline})

# Merge com os dados existentes, preenchendo gaps com NaN
df_ws100_filled = pd.merge(df_complete, df_ws100, on='DT', how='left')

print(f"\nShape do dataset após preenchimento: {df_ws100_filled.shape}")
print(f"Valores NaN na coluna SODAR-100: {df_ws100_filled['SODAR-100'].isna().sum()}")

Preenchendo gaps temporais com valores NaN...
Criando série temporal completa de 2021-11-10 17:20:00 até 2022-10-09 08:30:00...
Timestamps esperados: 47900
Timestamps existentes: 41786
Timestamps faltando: 6114

Shape do dataset após preenchimento: (47995, 2)
Valores NaN na coluna SODAR-100: 6921


/tmp/ipykernel_5653/3810799901.py:11: FutureWarning: 'T' is deprecated and will be removed in a future version, please use 'min' instead.
  complete_timeline = pd.date_range(start=start_time, end=end_time, freq='10T')


In [8]:
# Verificação da integridade dos dados preenchidos
print("Verificando integridade dos dados preenchidos...")

# Verificação se todos os intervalos são de 10 minutos
time_diffs_filled = df_ws100_filled['DT'].diff().dropna()
irregular_intervals = time_diffs_filled[time_diffs_filled != expected_interval]

print(f"Intervalos irregulares após preenchimento: {len(irregular_intervals)}")

if len(irregular_intervals) == 0:
    print("Sucesso: Todos os intervalos são de 10 minutos!")
else:
    print("Aviso: Ainda existem intervalos irregulares.")

# Estatísticas dos dados
total_points = len(df_ws100_filled)
missing_points = df_ws100_filled['SODAR-100'].isna().sum()
valid_points = total_points - missing_points

print(f"\nEstatísticas finais:")
print(f"Total de pontos: {total_points}")
print(f"Pontos válidos: {valid_points} ({valid_points/total_points*100:.1f}%)")
print(f"Pontos NaN: {missing_points} ({missing_points/total_points*100:.1f}%)")

Verificando integridade dos dados preenchidos...
Intervalos irregulares após preenchimento: 95
Aviso: Ainda existem intervalos irregulares.

Estatísticas finais:
Total de pontos: 47995
Pontos válidos: 41074 (85.6%)
Pontos NaN: 6921 (14.4%)


In [9]:
# Visualização de uma amostra dos dados preenchidos
print("Amostra dos dados preenchidos (primeiras 20 linhas):")
print(df_ws100_filled.head(20))

print("\nAmostra onde há valores NaN (primeiros 10):")
nan_samples = df_ws100_filled[df_ws100_filled['SODAR-100'].isna()].head(10)
if len(nan_samples) > 0:
    print(nan_samples)
else:
    print("Nenhum valor NaN encontrado nos primeiros registros.")

Amostra dos dados preenchidos (primeiras 20 linhas):
                    DT  SODAR-100
0  2021-11-10 17:20:00       5.83
1  2021-11-10 17:30:00       5.99
2  2021-11-10 17:40:00       6.67
3  2021-11-10 17:50:00       6.15
4  2021-11-10 18:00:00       5.90
5  2021-11-10 18:10:00       5.84
6  2021-11-10 18:20:00       5.80
7  2021-11-10 18:30:00       5.93
8  2021-11-10 18:40:00       6.22
9  2021-11-10 18:50:00       6.24
10 2021-11-10 19:00:00       6.15
11 2021-11-10 19:10:00       5.55
12 2021-11-10 19:20:00       6.53
13 2021-11-10 19:30:00       6.44
14 2021-11-10 19:40:00       6.61
15 2021-11-10 19:50:00       7.02
16 2021-11-10 20:00:00       7.38
17 2021-11-10 20:10:00       6.76
18 2021-11-10 20:20:00       7.07
19 2021-11-10 20:30:00       7.08

Amostra onde há valores NaN (primeiros 10):
                     DT  SODAR-100
351 2021-11-13 03:50:00        NaN
352 2021-11-13 04:00:00        NaN
353 2021-11-13 04:10:00        NaN
354 2021-11-13 04:20:00        NaN
355 2021-11-1

In [10]:
# Salvamento do dataset WS100 preenchido
print(f"Salvando dataset WS100 em {output_file}...")

# Criação do diretório de saída se não existir
output_file.parent.mkdir(exist_ok=True)

# Conversão da coluna DT de volta para string no formato original
df_ws100_filled['DT'] = df_ws100_filled['DT'].dt.strftime('%Y/%m/%d %H:%M:%S')

# Salvamento do arquivo CSV
df_ws100_filled.to_csv(output_file, index=False)

print(f"Dataset WS100 salvo com sucesso!")
print(f"Arquivo: {output_file}")
print(f"Tamanho: {output_file.stat().st_size / 1024:.1f} KB")

# Verificação final
print("\nVerificação final - carregando arquivo salvo:")
df_verify = pd.read_csv(output_file)
print(f"Shape: {df_verify.shape}")
print(f"Colunas: {list(df_verify.columns)}")
print(f"Primeiras 3 linhas:")
print(df_verify.head(3))

Salvando dataset WS100 em ../converted_csv_data/WS100.csv...
Dataset WS100 salvo com sucesso!
Arquivo: ../converted_csv_data/WS100.csv
Tamanho: 1147.0 KB

Verificação final - carregando arquivo salvo:
Shape: (47995, 2)
Colunas: ['DT', 'SODAR-100']
Primeiras 3 linhas:
                    DT  SODAR-100
0  2021/11/10 17:20:00       5.83
1  2021/11/10 17:30:00       5.99
2  2021/11/10 17:40:00       6.67
